In [13]:
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import *
bsr = bs(open("../data/jiading_map/Jiading.osm"),"xml")

nodes = bsr.find_all('node')
print(len(nodes))
# print nodes

4628


In [14]:
import pandas as pd
datalist=[]
for node in nodes:
    datalist.append([node['id'],node['lat'],node['lon']])
df1=pd.DataFrame(datalist,columns=['origin_id','lat','lon'])
df1['new_id']=df1.index
df1

,origin_id,lat,lon,new_id
0,368906068,31.2906170,121.2185829,0
1,499158740,31.2874178,121.1973003,1
2,499158746,31.2874739,121.1981567,2
3,499158754,31.2874220,121.1996538,3
4,499158759,31.2871701,121.2011454,4
5,499158764,31.2868806,121.2021130,5
6,499158768,31.2865055,121.2030370,6
7,499158775,31.2858191,121.2042977,7
8,499158780,31.2855488,121.2046891,8
9,499158783,31.2791592,121.2132000,9


In [4]:
print df1.lat.max()
print df1.lat.min()

31.4263578
31.2692027


In [5]:
print df1.lon.max()
print df1.lon.min()

121.2855157
121.1806807


In [15]:
df1.to_csv('../data/jiading_map/Nodes.csv',index=False)

In [44]:
# None sense...
# length=len(df1)
# file=open('WA_Nodes.txt','wb')
# for i in range(length):
#     print>>file,    df1.loc[i,'new_id'],df1.loc[i,'lat'],df1.loc[i,'lon']

# file.close()

In [16]:
ways = bsr.find_all('way')
way_kinds = ['motorway','motorway_link','primary','primary_link',\
            'secondary','secondary_link','tertiary', 'tertiary_link','residential','living_street',\
            'service','trunk','trunk_link','unclassified','road','track']

highways = []
way_types = set(['pedestrian', 'footway', 'steps'])
for w in ways:
    flag=False
    way_id = int(w.attrs['id'])
    for tag in w.find_all('tag'):
        if tag['k']=='highway':
            if tag['v'] in way_kinds and way_id not in [266205933, 266205935]:
                flag=True
                break
            else:
                if tag['v'] in way_types and way_id in [384531666, 384531667, 266221576, 263913992, 263913994, 135419990, 135372842, 135372843, 135372844, 135372845]:
                    flag=True
                    break
                
    if flag:
        highways.append(w)
# highways[:5]
print way_types, len(highways)

set(['pedestrian', 'steps', 'footway']) 143


In [18]:
import pandas as pd
from collections import defaultdict
segMap = defaultdict(int)

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n-1):
        yield l[i:i+n]
final_list=[]
for w in highways:

    isOneway = False
    category = 'unclassified'
    for tag in w.find_all('tag'):
        if tag['k']=='oneway' and tag['v'] =='yes':
            isOneway = True
        if tag['k']=='highway':
            category = tag['v']

    nodes=[nd['ref'] for nd in w.find_all('nd')]
    for node in nodes:
        segMap[node] += 1
    len_nodes=len(nodes)
    final_list.append([1, False,  category,nodes])
#     final_list.append([1, False,  category,nodes[::-1]])
#         nodes_list=list(chunks(nodes,4))
#         if(len(nodes_list[-1])==1):
#             nodes_list.pop()    
#         for nds in nodes_list:
#             final_list.append([1, isOneway,  category,nds])
junctions = set()       
for k in segMap.keys():
    if segMap[k] > 1:
        junctions.add(k)
    
print len(junctions)
df = pd.DataFrame(final_list,columns=['id','oneway','category','nodes'])
df

175


,id,oneway,category,nodes
0,1,False,trunk,"[1187045165, 1187044390]"
1,1,False,trunk,"[1187044733, 1797222842, 1187045164, 1187044863]"
2,1,False,trunk,"[1187044978, 1187044726, 1187061162, 118706119..."
3,1,False,trunk,"[1187044372, 1187044733]"
4,1,False,trunk,"[1187045195, 1187044507, 1187044881, 175100898..."
5,1,False,trunk,"[1187044863, 1187044978]"
6,1,False,trunk,"[1187044987, 1751008982, 1187045087, 118704477..."
7,1,False,trunk,"[499158740, 1187061213, 499158746, 1187061157,..."
8,1,False,primary,"[1187044372, 499158783, 1827278615, 1455069223..."
9,1,False,primary,"[1246571058, 1246571008]"


In [20]:
a = [1759597604, 3878436154, 2703796554]
list(reversed(a))

[2703796554L, 3878436154L, 1759597604]

In [21]:
final_list=[]
for index, row in df.iterrows():
#     print index, row['nodes']
    st = 0;
    node_list = []
    for i in range(1, len(row['nodes']) - 1):
        if row['nodes'][i] in junctions:
            node_list.append(row['nodes'][st:i+1])
            node_list.append(list(reversed(row['nodes'][st:i+1])))
            st = i
    node_list.append(row['nodes'][st:len(row['nodes'])])
    node_list.append(list(reversed(row['nodes'][st:len(row['nodes'])])))
    for nst in node_list:
        final_list.append([1, row['oneway'], row['category'], nst]) 
df = pd.DataFrame(final_list,columns=['id','oneway','category','nodes'])
df

,id,oneway,category,nodes
0,1,False,trunk,"[1187045165, 1187044390]"
1,1,False,trunk,"[1187044390, 1187045165]"
2,1,False,trunk,"[1187044733, 1797222842]"
3,1,False,trunk,"[1797222842, 1187044733]"
4,1,False,trunk,"[1797222842, 1187045164, 1187044863]"
5,1,False,trunk,"[1187044863, 1187045164, 1797222842]"
6,1,False,trunk,"[1187044978, 1187044726, 1187061162, 118706119..."
7,1,False,trunk,"[1187061183, 1187061223, 1187061253, 118706119..."
8,1,False,trunk,"[1187061183, 1187061211, 1187061243, 118706118..."
9,1,False,trunk,"[1187061255, 3459182336, 1187061236, 118706118..."


In [23]:
df['id']=df.index
df['nodes']=df['nodes'].apply(lambda x:list(x))
df

,id,oneway,category,nodes
0,0,False,trunk,"[1187045165, 1187044390]"
1,1,False,trunk,"[1187044390, 1187045165]"
2,2,False,trunk,"[1187044733, 1797222842]"
3,3,False,trunk,"[1797222842, 1187044733]"
4,4,False,trunk,"[1797222842, 1187045164, 1187044863]"
5,5,False,trunk,"[1187044863, 1187045164, 1797222842]"
6,6,False,trunk,"[1187044978, 1187044726, 1187061162, 118706119..."
7,7,False,trunk,"[1187061183, 1187061223, 1187061253, 118706119..."
8,8,False,trunk,"[1187061183, 1187061211, 1187061243, 118706118..."
9,9,False,trunk,"[1187061255, 3459182336, 1187061236, 118706118..."


In [24]:
df.to_csv('../data/jiading_map/highway.csv',index=False)